## Pre procesamiento de los datos

In [142]:
# importando librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [143]:
# cargar el dataset
datos = pd.read_csv('us_perm_visas.csv',low_memory=False,
        parse_dates=['decision_date','case_received_date'])

In [144]:
datos.head()

,add_these_pw_job_title_9089,agent_city,agent_firm_name,agent_state,application_type,case_no,case_number,case_received_date,case_status,class_of_admission,...,ri_pvt_employment_firm_to,ri_us_workers_considered,schd_a_sheepherder,us_economic_sector,wage_offer_from_9089,wage_offer_to_9089,wage_offer_unit_of_pay_9089,wage_offered_from_9089,wage_offered_to_9089,wage_offered_unit_of_pay_9089
0,NaN,NaN,NaN,NaN,PERM,A-07323-97014,NaN,NaT,Certified,J-1,...,NaN,NaN,NaN,IT,75629.0,NaN,yr,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,PERM,A-07332-99439,NaN,NaT,Denied,B-2,...,NaN,NaN,NaN,Other Economic Sector,37024.0,NaN,yr,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,PERM,A-07333-99643,NaN,NaT,Certified,H-1B,...,NaN,NaN,NaN,Aerospace,47923.0,NaN,yr,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,PERM,A-07339-01930,NaN,NaT,Certified,B-2,...,NaN,NaN,NaN,Other Economic Sector,10.97,NaN,hr,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,PERM,A-07345-03565,NaN,NaT,Certified,L-1,...,NaN,NaN,NaN,Advanced Mfg,100000.0,NaN,yr,NaN,NaN,NaN


### Pre-procesamiento de datos

In [145]:
# Número de atributos o características y número de filas
def getInfodata(data): 
    print("Número de atributos/columnas:",len(datos.columns))
    print("Número de filas:",len(datos))

In [146]:
# Porcentaje de valores faltantes por columna
def getOmisionColumnas(data):
    index=0
    for column in data.columns:
        index=index+1
        print(index,". El Attributo '",column,"' presenta ",  round(data[column].isnull().sum()*100/len(data),2), "% de valores nulos")

In [147]:
getInfodata(datos)

Número de atributos/columnas: 154
Número de filas: 374362


In [148]:
# Tipos de datos existentes en las características
datos.dtypes.value_counts()

object            142
float64            10
datetime64[ns]      2
dtype: int64

Todo el set de datos se compone de 374362 filas o registros de aplicaciones. Mientras que se cuentan con 154 atributos los cuales se dividen entre 142 valores categóricos, 10 numéricos y 2 de fechas

# Limpieza de datos y Evaluación de posible data duplicada o separada

In [149]:
print(datos.columns.values)

['add_these_pw_job_title_9089' 'agent_city' 'agent_firm_name'
 'agent_state' 'application_type' 'case_no' 'case_number'
 'case_received_date' 'case_status' 'class_of_admission'
 'country_of_citizenship' 'country_of_citzenship' 'decision_date'
 'employer_address_1' 'employer_address_2' 'employer_city'
 'employer_country' 'employer_decl_info_title' 'employer_name'
 'employer_num_employees' 'employer_phone' 'employer_phone_ext'
 'employer_postal_code' 'employer_state' 'employer_yr_estab'
 'foreign_worker_info_alt_edu_experience'
 'foreign_worker_info_birth_country' 'foreign_worker_info_city'
 'foreign_worker_info_education' 'foreign_worker_info_education_other'
 'foreign_worker_info_inst' 'foreign_worker_info_major'
 'foreign_worker_info_postal_code' 'foreign_worker_info_rel_occup_exp'
 'foreign_worker_info_req_experience' 'foreign_worker_info_state'
 'foreign_worker_info_training_comp' 'foreign_worker_ownership_interest'
 'foreign_worker_yr_rel_edu_completed' 'fw_info_alt_edu_experience'

# 1. case_number y case_no
donde la data que se encuentra en uno no se encuentra en la otra, con lo que serían complementarias

In [150]:
data=pd.DataFrame()
data['number']=datos['case_number']
data['no']=datos['case_no']
data['cantidad'] = data.apply(lambda x: x.count(), axis=1)

In [151]:
print('Total de registros: ',len(data))
data.loc[data['cantidad'] ==1].count()

Total de registros:  374362


number      239093
no          135269
cantidad    374362
dtype: int64

Se procederá a eliminar la data de ambas columnas y crear una nueva en función a la data de estos dos campos

In [152]:
datos['case_number_new']=datos['case_number']
datos['case_number_new']
mask = datos['case_number_new'].isnull()
column_name = 'case_number_new'
datos.loc[mask, column_name] = datos['case_no']


In [153]:
datos.drop(['case_number','case_no'], axis=1,inplace=True)
getInfodata(datos)

Número de atributos/columnas: 153
Número de filas: 374362


# 2. case_status
Dentro del modelo se identifican 4 tipos de case_status. Para el presente experimento, se tomarán solo los casos con status Certified y Denied. Para los casos de Withdran, estos no serán considerados, por lo cual serán retirados del dataset. Para los casos de Certified-Expired, debido a la información revisada, esots corresponden a casos que una vez obtevieron el estado Certified, debieron presentar el formulario I-140 en los siguientes 6 meses. Pasado este tiempo, se procede al cambio del estado. En ese sentido, este grupo será considerado como Certified.

In [154]:
datos.case_status.value_counts()

Certified            181933
Certified-Expired    148586
Denied                25649
Withdrawn             18194
Name: case_status, dtype: int64

In [155]:
datos = datos[datos.case_status != 'Withdrawn']
datos.case_status.value_counts()

Certified            181933
Certified-Expired    148586
Denied                25649
Name: case_status, dtype: int64

In [156]:
datos.loc[datos['case_status'] == 'Certified-Expired', 'case_status'] = 'Certified'
datos.case_status.value_counts()

Certified    330519
Denied        25649
Name: case_status, dtype: int64

# 3. filas y columnas nulas
donde la data que se encuentra en uno no se encuentra en la otra, con lo que serían complementarias

In [157]:
print("Datos iniciales:\n")
getInfodata(datos)
datos = datos.dropna(axis=0, how='all');
datos = datos.dropna(axis=1, how='all');
print("\nDatos finales:\n")
getInfodata(datos)

Datos iniciales:

Número de atributos/columnas: 153
Número de filas: 356168

Datos finales:

Número de atributos/columnas: 153
Número de filas: 356168


No existen filas ni columnas con todos los valores nulos

# 4. columnas con más del 85% de omisiones
Se define como porcentaje mínimo para considerar un atributo que tenga al menos el 85% de sus filas con información

In [158]:
# Porcentaje de omisiones de cada columna
getOmisionColumnas(datos)

1 . El Attributo ' add_these_pw_job_title_9089 ' presenta  89.01 % de valores nulos
2 . El Attributo ' agent_city ' presenta  43.08 % de valores nulos
3 . El Attributo ' agent_firm_name ' presenta  44.26 % de valores nulos
4 . El Attributo ' agent_state ' presenta  43.95 % de valores nulos
5 . El Attributo ' application_type ' presenta  64.39 % de valores nulos
6 . El Attributo ' case_received_date ' presenta  35.61 % de valores nulos
7 . El Attributo ' case_status ' presenta  0.0 % de valores nulos
8 . El Attributo ' class_of_admission ' presenta  5.92 % de valores nulos
9 . El Attributo ' country_of_citizenship ' presenta  5.41 % de valores nulos
10 . El Attributo ' country_of_citzenship ' presenta  94.6 % de valores nulos
11 . El Attributo ' decision_date ' presenta  0.0 % de valores nulos
12 . El Attributo ' employer_address_1 ' presenta  0.01 % de valores nulos
13 . El Attributo ' employer_address_2 ' presenta  39.6 % de valores nulos
14 . El Attributo ' employer_city ' presenta  

105 . El Attributo ' recr_info_coll_univ_teacher ' presenta  35.62 % de valores nulos
106 . El Attributo ' recr_info_employer_rec_payment ' presenta  35.62 % de valores nulos
107 . El Attributo ' recr_info_first_ad_start ' presenta  37.29 % de valores nulos
108 . El Attributo ' recr_info_job_fair_from ' presenta  99.52 % de valores nulos
109 . El Attributo ' recr_info_job_fair_to ' presenta  99.52 % de valores nulos
110 . El Attributo ' recr_info_on_campus_recr_from ' presenta  99.5 % de valores nulos
111 . El Attributo ' recr_info_on_campus_recr_to ' presenta  99.51 % de valores nulos
112 . El Attributo ' recr_info_pro_org_advert_from ' presenta  96.27 % de valores nulos
113 . El Attributo ' recr_info_pro_org_advert_to ' presenta  96.27 % de valores nulos
114 . El Attributo ' recr_info_prof_org_advert_from ' presenta  92.31 % de valores nulos
115 . El Attributo ' recr_info_prof_org_advert_to ' presenta  92.31 % de valores nulos
116 . El Attributo ' recr_info_professional_occ ' present

In [159]:
# Vamos a filtrar solo aquellas columnas que tienen al menos un 85% de valores no perdidos
numPermitido = len(datos)*0.85
datos = datos.loc[:, datos.isnull().sum() <= numObservaciones]
datos.shape

(356168, 120)

In [160]:
getOmisionColumnas(datos)

1 . El Attributo ' agent_city ' presenta  43.08 % de valores nulos
2 . El Attributo ' agent_firm_name ' presenta  44.26 % de valores nulos
3 . El Attributo ' agent_state ' presenta  43.95 % de valores nulos
4 . El Attributo ' application_type ' presenta  64.39 % de valores nulos
5 . El Attributo ' case_received_date ' presenta  35.61 % de valores nulos
6 . El Attributo ' case_status ' presenta  0.0 % de valores nulos
7 . El Attributo ' class_of_admission ' presenta  5.92 % de valores nulos
8 . El Attributo ' country_of_citizenship ' presenta  5.41 % de valores nulos
9 . El Attributo ' decision_date ' presenta  0.0 % de valores nulos
10 . El Attributo ' employer_address_1 ' presenta  0.01 % de valores nulos
11 . El Attributo ' employer_address_2 ' presenta  39.6 % de valores nulos
12 . El Attributo ' employer_city ' presenta  0.0 % de valores nulos
13 . El Attributo ' employer_country ' presenta  35.63 % de valores nulos
14 . El Attributo ' employer_decl_info_title ' presenta  35.63 % d

107 . El Attributo ' ri_job_search_website_from ' presenta  46.71 % de valores nulos
108 . El Attributo ' ri_job_search_website_to ' presenta  46.72 % de valores nulos
109 . El Attributo ' ri_layoff_in_past_six_months ' presenta  35.62 % de valores nulos
110 . El Attributo ' ri_local_ethnic_paper_from ' presenta  67.34 % de valores nulos
111 . El Attributo ' ri_local_ethnic_paper_to ' presenta  67.34 % de valores nulos
112 . El Attributo ' ri_posted_notice_at_worksite ' presenta  35.64 % de valores nulos
113 . El Attributo ' schd_a_sheepherder ' presenta  35.62 % de valores nulos
114 . El Attributo ' us_economic_sector ' presenta  66.06 % de valores nulos
115 . El Attributo ' wage_offer_from_9089 ' presenta  30.24 % de valores nulos
116 . El Attributo ' wage_offer_to_9089 ' presenta  80.13 % de valores nulos
117 . El Attributo ' wage_offer_unit_of_pay_9089 ' presenta  30.54 % de valores nulos
118 . El Attributo ' wage_offered_from_9089 ' presenta  69.79 % de valores nulos
119 . El Attr

# 5. columnas derivadas
En función a la información brindada, se generarán nuevas columnas y se estandarizarán las que correspondan

In [161]:
datos['year'] = datos['decision_date'].dt.year

In [162]:
datos['employer_city'] = datos['employer_city'].str.upper()
datos['employer_city'].value_counts().head(12)

NEW YORK           17198
COLLEGE STATION    11985
SANTA CLARA        10519
SAN JOSE            9147
REDMOND             8485
MOUNTAIN VIEW       8121
HOUSTON             6720
SAN FRANCISCO       6352
SUNNYVALE           6104
PLANO               5607
CHICAGO             5561
SEATTLE             5051
Name: employer_city, dtype: int64

In [163]:
#Estandarización de los cargos
datos['job_info_job_title'].value_counts()[:30]

Software Engineer                   14372
Computer Systems Analyst - V         4747
Senior Software Engineer             3801
Computer Systems Analyst - II        3416
Software Developer                   2880
Assistant Professor                  2604
Programmer Analyst                   2534
Computer Systems Analyst - IV        2496
Systems Analyst                      2034
Software Development Engineer II     1630
Poultry Processing Worker            1594
Component Design Engineer            1064
Poultry Processor                    1037
Business Analyst                      963
software engineer                     932
SOFTWARE ENGINEER                     877
Senior Programmer Analyst             862
Sr. Software Engineer                 847
Senior Software Developer             817
Commercial Cleaner                    802
Process Engineer                      746
Accountant                            719
Computer Programmer                   687
Computer Systems Analyst          

In [164]:
datos['job_info_job_title'] = datos['job_info_job_title'].str.lower()
datos['job_info_job_title'] = datos['job_info_job_title'].astype(str).str.split('-').str[0]
datos['job_info_job_title'] = datos['job_info_job_title'].astype(str).str.split('ii').str[0]
datos['job_info_job_title'] = datos['job_info_job_title'].astype(str).str.split('/').str[0]
datos['job_info_job_title'] = datos['job_info_job_title'].astype(str).str.strip()
datos['job_info_job_title'] = datos['job_info_job_title'].str.replace('sr.', 'senior')
datos['job_info_job_title'].replace(["NaN", 'NaT','nan'], np.nan, inplace = True)
datos['job_info_job_title'].value_counts()[:20]

software engineer                18582
computer systems analyst         12054
senior software engineer          5802
software developer                4501
programmer analyst                3763
assistant professor               2869
software development engineer     2766
systems analyst                   2587
senior programmer analyst         1884
senior software developer         1625
poultry processing worker         1596
business analyst                  1533
poultry processor                 1153
project manager                   1066
component design engineer         1064
accountant                         899
commercial cleaner                 846
database administrator             838
senior systems analyst             835
computer programmer                813
Name: job_info_job_title, dtype: int64

In [165]:
# estandarizacion de los sueldos y unidades de aplicacion del sueldo
datos[['pw_amount_9089','pw_unit_of_pay_9089']].head(20)

,pw_amount_9089,pw_unit_of_pay_9089
0,75629.0,yr
1,37024.0,yr
2,47923.0,yr
3,10.97,hr
4,94890.0,yr
5,37024.0,yr
6,47083.33,yr
7,36733.0,yr
8,44824.0,yr
9,12.86,hr


In [166]:
datos['pw_amount_9089'] = datos['pw_amount_9089'].str.replace(",","") 

for unit in datos.pw_unit_of_pay_9089.unique():
    if unit == "hr" or unit == "Hour":
        datos.loc[datos['pw_unit_of_pay_9089'] == unit, 'pw_amount_9089'] = datos['pw_amount_9089'].apply(lambda x: float(x) * 8 * 250)
        datos.loc[datos['pw_unit_of_pay_9089'] == unit, 'pw_unit_of_pay_9089'] = datos['pw_unit_of_pay_9089'].replace(to_replace = unit, value = "Year") 
    elif unit == "wk" or unit == "Week":
        datos.loc[datos['pw_unit_of_pay_9089'] == unit, 'pw_amount_9089'] = datos['pw_amount_9089'].apply(lambda x: float(x) * 50)
        datos.loc[datos['pw_unit_of_pay_9089'] == unit, 'pw_unit_of_pay_9089'] = datos['pw_unit_of_pay_9089'].replace(to_replace = unit, value = "Year")
    elif unit == "mth" or unit == "Month":
        datos.loc[datos['pw_unit_of_pay_9089'] == unit, 'pw_amount_9089'] = datos['pw_amount_9089'].apply(lambda x: float(x) * 12)
        datos.loc[datos['pw_unit_of_pay_9089'] == unit, 'pw_unit_of_pay_9089'] = datos['pw_unit_of_pay_9089'].replace(to_replace = unit, value = "Year")
    elif unit == "bi" or unit == "Bi-Weekly":  
        datos.loc[datos['pw_unit_of_pay_9089'] == unit, 'pw_amount_9089'] = datos['pw_amount_9089'].apply(lambda x: float(x) * 25)
        datos.loc[datos['pw_unit_of_pay_9089'] == unit, 'pw_unit_of_pay_9089'] = datos['pw_unit_of_pay_9089'].replace(to_replace = unit, value = "Year")
    elif unit =="yr":
         datos.loc[datos['pw_unit_of_pay_9089'] == unit ,'pw_unit_of_pay_9089'] = datos['pw_unit_of_pay_9089'].replace(to_replace = unit, value = "Year")
    else:
        continue
        
#Reemplazamos los valores con la media
datos['pw_amount_9089']= datos['pw_amount_9089'].fillna((datos['pw_amount_9089'].median()))

#Se formatea el valor de los montos
datos['pw_amount_9089'] = datos.pw_amount_9089.astype(float)

datos[['pw_amount_9089','pw_unit_of_pay_9089']].head(20)

,pw_amount_9089,pw_unit_of_pay_9089
0,75629.00,Year
1,37024.00,Year
2,47923.00,Year
3,21940.00,Year
4,94890.00,Year
5,37024.00,Year
6,47083.33,Year
7,36733.00,Year
8,44824.00,Year
9,25720.00,Year


In [167]:
with pd.option_context('float_format', '{:.2f}'.format): print(datos.pw_amount_9089.describe())

count      356168.00
mean       469624.51
std       7679620.28
min             7.50
25%         67808.00
50%         86278.00
75%        104790.00
max     495748000.00
Name: pw_amount_9089, dtype: float64


In [168]:
# agrupamos en categorias los salarios para una mayor facilidad
datos['remuneration'] = pd.cut(datos['pw_amount_9089'], [0, 30000, 60000,90000,120000,150000,180000,210000,240000,270000,495748000], right=False, labels=["0-30k", "30-60k","60-90k","90-120k","120-150k","150-180k","180-210k","210-240k","240-270k","270k+"])
salary = datos['remuneration'].value_counts()
salary.iloc[np.argsort(salary.index)]

0-30k        19810
30-60k       47493
60-90k      133050
90-120k     112631
120-150k     30532
150-180k      7094
180-210k      3801
210-240k       604
240-270k        29
270k+         1123
Name: remuneration, dtype: int64

In [169]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356168 entries, 0 to 374353
Columns: 122 entries, agent_city to remuneration
dtypes: category(1), datetime64[ns](2), float64(9), int64(1), object(109)
memory usage: 341.9+ MB


In [170]:
#Reemplazamos los valores de case_status por 1 para certified y 0 para denied
datos.loc[datos.case_status == 'Certified', 'case_status'] = 1
datos.loc[datos.case_status == 'Denied', 'case_status'] = 0
datos['case_status'] = datos['case_status'].astype(int)


#Completamos el estado con la moda
datos['employer_state'] = datos['employer_state'].fillna(datos['employer_state'].mode()[0]);

In [171]:
# mapeamos los estados
state_abbrevs = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
    'Northern Mariana Islands':'MP', 
    'Palau': 'PW', 
    'Puerto Rico': 'PR', 
    'Virgin Islands': 'VI', 
    'District of Columbia': 'DC'
}

us_state_abbrev = {k.upper(): v for k, v in state_abbrevs.items()}
datos['employer_state'].replace(us_state_abbrev, inplace=True)
datos.employer_state = datos.employer_state.astype(str)


In [172]:
datos['pw_soc_code'] = datos['pw_soc_code'].str.replace('.','')
datos['pw_soc_code'] = datos['pw_soc_code'].str.replace('-','')
datos['pw_soc_code'] = datos['pw_soc_code'].astype(str).str[0:6]
datos['pw_soc_code'].value_counts()

#reemplazamos el pw_soc_code por la moda
datos.loc[datos['pw_soc_code'] == "nan",'pw_soc_code'] = datos['pw_soc_code'].mode()[0]

#reemplazamos los none por la moda
datos.loc[datos['pw_soc_code'] == "None",'pw_soc_code'] = datos['pw_soc_code'].mode()[0]

datos['pw_soc_code'] = datos['pw_soc_code'].astype(int)

In [173]:
datos['class_of_admission']=datos['class_of_admission'].fillna((datos['class_of_admission'].mode()[0]))
datos['country_of_citizenship']=datos['country_of_citizenship'].fillna((datos['country_of_citizenship'].mode()[0]))
datos['employer_city']=datos['employer_city'].fillna((datos['employer_city'].mode()[0]))
datos['pw_soc_title']=datos['pw_soc_title'].fillna((datos['pw_soc_title'].mode()[0]))
datos['job_info_work_city']=datos['job_info_work_city'].fillna((datos['job_info_work_city'].mode()[0]))
datos['job_info_work_state']=datos['job_info_work_state'].fillna((datos['job_info_work_state'].mode()[0]))
datos['pw_level_9089']=datos['pw_level_9089'].fillna((datos['pw_level_9089'].mode()[0]))
datos['pw_unit_of_pay_9089']=datos['pw_unit_of_pay_9089'].fillna((datos['pw_unit_of_pay_9089'].mode()[0]))
datos['employer_name']=datos['employer_name'].fillna((datos['employer_name'].mode()[0]))
datos['employer_name']=datos['employer_name'].astype(str).str.upper()
datos['pw_source_name_9089']=datos['pw_source_name_9089'].fillna((datos['pw_source_name_9089'].mode()[0]))
datos['remuneration']=datos['remuneration'].fillna((datos['remuneration'].mode()[0]))

In [174]:
# Vamos a filtrar solo aquellas columnas que tienen al menos un 85% de valores no perdidos
numPermitido = len(datos)*0.10
datos = datos.loc[:, datos.count() >= numObservaciones]
datos.shape

(356168, 20)

In [175]:
getOmisionColumnas(datos)

1 . El Attributo ' case_status ' presenta  0.0 % de valores nulos
2 . El Attributo ' class_of_admission ' presenta  0.0 % de valores nulos
3 . El Attributo ' country_of_citizenship ' presenta  0.0 % de valores nulos
4 . El Attributo ' decision_date ' presenta  0.0 % de valores nulos
5 . El Attributo ' employer_address_1 ' presenta  0.01 % de valores nulos
6 . El Attributo ' employer_city ' presenta  0.0 % de valores nulos
7 . El Attributo ' employer_name ' presenta  0.0 % de valores nulos
8 . El Attributo ' employer_postal_code ' presenta  0.01 % de valores nulos
9 . El Attributo ' employer_state ' presenta  0.0 % de valores nulos
10 . El Attributo ' job_info_work_city ' presenta  0.0 % de valores nulos
11 . El Attributo ' job_info_work_state ' presenta  0.0 % de valores nulos
12 . El Attributo ' pw_amount_9089 ' presenta  0.0 % de valores nulos
13 . El Attributo ' pw_level_9089 ' presenta  0.0 % de valores nulos
14 . El Attributo ' pw_soc_code ' presenta  0.0 % de valores nulos
15 . E